## Introduction

In this Notebook we will analysis the data and show it on the map

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import datetime
import folium
import plotly
import os

from shapely import wkt
from plotly.subplots import make_subplots
from folium.plugins import HeatMap, MarkerCluster

## Import data

read data and convert the date columns to datetime

In [ ]:
date_parser = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

airport_df = pd.read_csv('/kaggle/input/covid19s-impact-on-airport-traffic/covid_impact_on_airport_traffic.csv', parse_dates=['Date'], date_parser=date_parser)
airport_df.head()

In [ ]:
airport_df.info()

## Data Visualization

Here, we consider a DataFrame having coordinates in WKT(well-known text) format

In [ ]:
airport_df['Centroid'] = airport_df['Centroid'].apply(wkt.loads)
airport_df['Geography'] = airport_df['Geography'].apply(wkt.loads)

Convert dataframe to geodataframe

In [ ]:
airport_geo = gpd.GeoDataFrame(airport_df, geometry= 'Centroid')
airport_geo.head()

iso types

In [ ]:
iso_type = airport_geo.ISO_3166_2.value_counts(ascending=True)
px.bar(y=iso_type,
      x=iso_type.index,
      color=iso_type)

airports with the most data

In [ ]:
# The top records of the airport
airport_rec = airport_geo.AirportName.value_counts(ascending=True)
px.bar(y=airport_rec,
      x=airport_rec.index,
      color=airport_rec)

countries with the most data

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
airport_geo.Country.value_counts(ascending=True).plot(kind='barh')

plt.subplot(1,2,2)
airport_geo.Country.value_counts().plot(kind='pie', legend=True)

averange of  baseline percentage by month

In [ ]:
baseline_df = airport_geo[['Date', 'PercentOfBaseline']]
baseline_df['month'] = airport_geo.Date.apply(lambda x: x.month_name())
baseline_df['day'] = airport_geo.Date.apply(lambda x: x.day_name())

month_percent = baseline_df.groupby('month').PercentOfBaseline.agg('mean')
day_percent = baseline_df.groupby('day').PercentOfBaseline.agg('mean')

In [ ]:
fig_bar = make_subplots(rows=1, cols=2, start_cell='bottom-left', subplot_titles=('Month', 'Day'))

fig_bar.add_trace(
    go.Bar(x=month_percent.index, y=month_percent, name='Bar'), row=1, col=1
)

fig_bar.add_trace(
    go.Scatter(x=month_percent.index, y=month_percent, name='Scatter'), row=1, col=1
)

fig_bar.add_trace(
    go.Bar(x=day_percent.index, y=day_percent, name='Bar'), row=1, col=2
)

fig_bar.add_trace(
    go.Scatter(x=day_percent.index, y=day_percent, name='Scatter'), row=1, col=2
)

fig_bar.update_layout(height=400, title='baseline percentage by month and day', showlegend=False)

fig_bar.show()

In [ ]:
def return_pull(value):
    return [0.2 if v==value.max() else 0  for v in value]

fig_pie = make_subplots(rows=1, cols=2, subplot_titles=['Month', 'Day'], specs=[[{'type': 'domain'}, {'type': 'domain'}]])

fig_pie.add_trace(
    go.Pie(values=month_percent, labels=month_percent.index, name='Month', pull=return_pull(month_percent)), 1,1
)

fig_pie.add_trace(
    go.Pie(values=day_percent, labels=day_percent.index, name='Day', pull=return_pull(day_percent)), 1,2
)

fig_pie.update_layout(
    title='baseline percentage by month and day',
    showlegend=False,
    height=400
)

fig_pie.show()

In [ ]:
airport_xy = airport_geo[~airport_geo.AirportName.duplicated()][['AirportName', 'Centroid', 'PercentOfBaseline']].reset_index(drop=True)

# obtain the latitude and logitude from data
airport_xy['lat'] = airport_xy.Centroid.y
airport_xy['long'] = airport_xy.Centroid.x
print(airport_xy.shape)
airport_xy.head()

Show on the map of the planet

In [ ]:
fig_map = go.Figure(data=[
    go.Scattergeo(lat=airport_xy['lat'],
             lon=airport_xy['long'],
             mode='markers',
             hoverinfo='text',
             marker=dict(color="darkred", size=5, opacity=0.9))
])
fig_map.update_geos(
    projection_type='orthographic',
    landcolor='black',
    showocean=True,
    oceancolor='MidnightBlue',
    lakecolor='blue',
)
fig_map.update_layout(
    width=800,
    height=400,
    template="plotly_white",
    margin=dict(r=0, t=0, b=0, l=0),
    annotations=[
        dict(
            text="",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ])
fig_map.show()

In [ ]:
class GeoMap:
    def __init__(self, geodata):
        assert type(geodata) == gpd.GeoDataFrame, 'geodata is not a GeoDataFrame: {}'.format(type(geodata))
        
        self.geo_df = geodata
    
    def heat_map(self, radius, location=None, zoom=None):
        assert type(radius) == int, 'radius is not a interger value: {}'.format(type(radius))
        assert type(location) == list, 'location is not a list: {}'.format(type(location))
        assert type(zoom) == int, 'zoom is not a interger value: {}'.format(type(zoom))
        
        try:
            map_ = folium.Map(location=location, zoom_start=zoom)
            HeatMap(data=self.geo_df[['lat', 'long']], radius=radius).add_to(map_)
        
            return map_
        
        except Exception as e:
            print(e)
            
    def marker_map(self, location=None, zoom=None):
        assert type(location) == list, 'location is not a list: {}'.format(type(location))
        assert type(zoom) == int, 'zoom is not a interger value: {}'.format(type(zoom))
        
        try:
            map_ = folium.Map(location=location, zoom_start=zoom)
            
            m_cluster = MarkerCluster()
            for idx, row in self.geo_df.iterrows():
                m_cluster.add_child(folium.Marker([row['lat'], row['long']],
                            popup='airport: {}\nlatitude: {:.2f}\nlogitude: {:.2f}'.format(row['AirportName'], row['lat'], row['long']),
                            icon=folium.Icon(icon='plane', prefix='fa')))
            return map_.add_child(m_cluster)
        
        except Exception as e:
            print(e)
            
    def circle_map(self, radius, location=None, zoom=None):
        assert type(radius) == int, 'radius is not a interger value: {}'.format(type(radius))
        assert type(location) == list, 'location is not a list: {}'.format(type(location))
        assert type(zoom) == int, 'zoom is not a interger value: {}'.format(type(zoom))

        try:
            COLOR = lambda x: 'forestgreen' if x>50 else 'darkred'

            map_ = folium.Map(location=location, zoom_start=zoom)
            for idx, row in self.geo_df.iterrows():
                folium.Circle(
                    location=[self.geo_df.iloc[idx]['lat'], self.geo_df.iloc[idx]['long']],
                    radius=radius,
                    color=COLOR(self.geo_df.iloc[idx]['PercentOfBaseline'])
                ).add_to(map_)

            return map_
        
        except Exception as e:
            print(e)

Each marker below corresponds to a different records

In [ ]:
geomap = GeoMap(airport_xy)

geomap.marker_map([30, 31], 1)

This shows the density of records in different regions of the countries

In [ ]:
geomap.heat_map(10, [30, 31], 1)

If the percentage of the baseline is more than 50, the color of the circle in the map turns green otherwise it turns red

In [ ]:
geomap.circle_map(10, [30, 31], 1)